In [1]:
from io import StringIO
import boto3
import pandas as pd
import numpy as np
from datetime import datetime

s3 = boto3.client("s3")

def list_csv_files(bucket_name, key_path):
    csv_files = []
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=key_path)
    
    for page in page_iterator:
        if 'Contents' in page:
            for content in page['Contents']:
                if content['Key'].endswith('.csv'):
                    csv_files.append(content['Key'])
    
    return csv_files

def read_csv_files_to_dataframes(bucket_name, csv_files):
    dataframes = []
    for key in csv_files:
        # Get the object from S3
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        # Read the CSV file content
        data = obj['Body'].read().decode('utf-8')
        # Convert to DataFrame
        df = pd.read_csv(StringIO(data))
        dataframes.append(df)
    return dataframes

def write_csv_to_s3(df, bucket_name, output_key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3.put_object(Bucket=bucket_name, Key=output_key, Body=csv_buffer.getvalue())
    print(f"Uploaded to s3://{bucket_name}/{output_key}")

In [2]:
bucket_name = 'niwa-water-demand-modelling'
key_path = 'TransformedOutputs/InferenceData/'
target_files = list_csv_files(bucket_name, key_path)
input_files = list_csv_files(bucket_name, "InferenceData/")

In [3]:
auto_ml_job_dict = {
    'NorthWellingtonMoa': 'Canvas1734649444174',
    'WellingtonLowLevel': 'Canvas1734648978161',
    'Petone': 'Canvas1733434154045',
    'WellingtonHighWestern': 'Canvas1733085655509',
    'WellingtonHighMoa': 'Canvas1733372214860',
    'NorthWellingtonPorirua': 'Canvas1733369877242',
    'Porirua': 'Canvas1733437572452',
    'Wainuiomata': 'Canvas1734649248674',
    'UpperHutt': 'Canvas1734649294393',
    'LowerHutt': 'Canvas1734649384856'
}

for key in list(auto_ml_job_dict.keys()):
    key = f"/{key}/"
    key_inputs = [e for e in input_files if key in e]
    key_files = ["/".join(e.split("/")[1:]) for e in target_files if key in e]
    unfinished = [e for e in key_inputs if e not in key_files]
    print(f"{key}: {len(key_files)}")
    # find out which input file is not covered
    print(f"{key}: {len(unfinished)} files not processed: {unfinished}")

/NorthWellingtonMoa/: 1
/NorthWellingtonMoa/: 0 files not processed: []
/WellingtonLowLevel/: 1
/WellingtonLowLevel/: 0 files not processed: []
/Petone/: 1
/Petone/: 0 files not processed: []
/WellingtonHighWestern/: 1
/WellingtonHighWestern/: 0 files not processed: []
/WellingtonHighMoa/: 1
/WellingtonHighMoa/: 0 files not processed: []
/NorthWellingtonPorirua/: 1
/NorthWellingtonPorirua/: 0 files not processed: []
/Porirua/: 1
/Porirua/: 0 files not processed: []
/Wainuiomata/: 1
/Wainuiomata/: 0 files not processed: []
/UpperHutt/: 1
/UpperHutt/: 0 files not processed: []
/LowerHutt/: 1
/LowerHutt/: 0 files not processed: []


In [4]:
target_files

['TransformedOutputs/InferenceData/LowerHutt/Lower Hutt.csv',
 'TransformedOutputs/InferenceData/NorthWellingtonMoa/North Wellington Moa.csv',
 'TransformedOutputs/InferenceData/NorthWellingtonPorirua/North Wellington Porirua.csv',
 'TransformedOutputs/InferenceData/Petone/Petone.csv',
 'TransformedOutputs/InferenceData/Porirua/Porirua.csv',
 'TransformedOutputs/InferenceData/UpperHutt/Upper Hutt.csv',
 'TransformedOutputs/InferenceData/Wainuiomata/Wainuiomata.csv',
 'TransformedOutputs/InferenceData/WellingtonHighMoa/Wellington High Moa.csv',
 'TransformedOutputs/InferenceData/WellingtonHighWestern/Wellington High Western.csv',
 'TransformedOutputs/InferenceData/WellingtonLowLevel/Wellington Low Level.csv']

In [5]:
input_files

['InferenceData/LowerHutt/Lower Hutt.csv',
 'InferenceData/NorthWellingtonMoa/North Wellington Moa.csv',
 'InferenceData/NorthWellingtonPorirua/North Wellington Porirua.csv',
 'InferenceData/Petone/Petone.csv',
 'InferenceData/Porirua/Porirua.csv',
 'InferenceData/UpperHutt/Upper Hutt.csv',
 'InferenceData/Wainuiomata/Wainuiomata.csv',
 'InferenceData/WellingtonHighMoa/Wellington High Moa.csv',
 'InferenceData/WellingtonHighWestern/Wellington High Western.csv',
 'InferenceData/WellingtonLowLevel/Wellington Low Level.csv']

In [31]:
y_cols = ['Lower Hutt', 'Petone',
       'Wainuiomata', 'Upper Hutt', 'Porirua', 'Wellington High Moa',
       'Wellington High Western', 'Wellington Low Level',
       'North Wellington Moa', 'North Wellington Porirua']

# find out unique experiments
experiments = ["InferenceData"]

for exp in experiments:
    exp_files = [e for e in target_files if exp in e]
    exp_input_files = [e for e in input_files if exp in e]
    ordered_files = []
    df_list = []
    # process by site name
    for y_col in y_cols:
        # find input file
        input_file = [e for e in exp_input_files if f"/{y_col}.csv" in e]
        # find prediction file from this experiment
        target_file = [e for e in exp_files if f"/{y_col}.csv" in e]
        # find historical pred
        y_col_1 = y_col.replace(" ", "")
        hp_file = [f"HistoricalData/{y_col_1}_historical_pred.csv"]
        df_hp = read_csv_files_to_dataframes(bucket_name, hp_file)[0]
        # find training pred
        tp_file = [f"HistoricalData/{y_col_1}_training_pred.csv"]
        df_tp = read_csv_files_to_dataframes(bucket_name, tp_file)[0]
        if input_file and target_file:
            df_input = read_csv_files_to_dataframes(bucket_name, input_file)[0]
            df_target = read_csv_files_to_dataframes(bucket_name, target_file)[0]
            df = pd.concat([df_input[["Date"]], df_target[[y_col]]], axis=1)
            df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
            df_hp["Date"] = pd.to_datetime(df_hp["Date"], format='mixed', dayfirst=True)
            df_tp["Date"] = pd.to_datetime(df_tp["Date"], format="%Y-%m-%d")
            df  = df[["Date", y_col]].set_index("Date")
            df_1 = pd.concat(
                [
                    df_hp[["Date", y_col]].set_index("Date"),
                    df_tp[["Date", y_col]].set_index("Date"),
                    df
                ], axis=0
            )
                
            df_list.append(df_1)
            print(f"{exp}: retrieved {y_col} results")
    df_ps = pd.concat(df_list, axis=1)
    output_key = f"InferenceResults/full_results.csv"
    write_csv_to_s3(df_ps, bucket_name, output_key)

InferenceData: retrieved Lower Hutt results
InferenceData: retrieved Petone results
InferenceData: retrieved Wainuiomata results
InferenceData: retrieved Upper Hutt results
InferenceData: retrieved Porirua results
InferenceData: retrieved Wellington High Moa results
InferenceData: retrieved Wellington High Western results
InferenceData: retrieved Wellington Low Level results
InferenceData: retrieved North Wellington Moa results
InferenceData: retrieved North Wellington Porirua results
Uploaded to s3://niwa-water-demand-modelling/InferenceResults/full_results.csv


In [32]:
df_ps

,Lower Hutt,Petone,Wainuiomata,Upper Hutt,Porirua,Wellington High Moa,Wellington High Western,Wellington Low Level,North Wellington Moa,North Wellington Porirua
Date,,,,,,,,,,
1890-01-01,205.902985,210.382324,183.672028,194.201157,181.258011,284.024384,290.420593,238.344803,161.833832,170.919006
1890-01-02,224.046783,267.977203,207.876556,183.547073,195.636185,270.371857,271.847351,266.280731,180.668991,186.806702
1890-01-03,252.940292,228.019455,185.308365,212.807022,194.083252,255.836502,247.523880,258.520325,195.777573,196.248917
1890-01-04,257.139191,246.623642,218.301422,246.866318,219.326019,224.410217,210.442291,273.646484,225.519669,199.940109
1890-01-05,247.511734,227.815552,224.862411,231.031586,215.086563,219.480179,210.258072,260.815125,217.679794,209.409607
...,...,...,...,...,...,...,...,...,...,...
2025-02-21,224.888412,268.098602,180.172684,214.198074,182.081894,278.246216,285.659424,261.230164,174.509308,176.042648
2025-02-22,256.141113,278.467224,213.739532,243.120239,217.707062,273.682251,259.437927,287.485931,223.184113,220.377625
2025-02-23,250.420868,274.420197,223.911240,255.959473,229.099609,245.479095,233.506577,271.510956,230.538086,228.087219


## clean-up transformed outputs

In [22]:
# s3 = boto3.client('s3')

# def delete_files_with_pattern(bucket_name, prefix, suffix):
#     paginator = s3.get_paginator('list_objects_v2')
#     page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    
#     for page in page_iterator:
#         if 'Contents' in page:
#             for content in page['Contents']:
#                 if content['Key'].endswith(suffix):
#                     print(f"Deleting {content['Key']}")
#                     s3.delete_object(Bucket=bucket_name, Key=content['Key'])

# bucket_name = 'niwa-water-demand-modelling'
# prefix = 'TransformedOutputs/Simulation/'
# suffix = '.csv.out'
# delete_files_with_pattern(bucket_name, prefix, suffix)

Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_0.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_1.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_2.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_3.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_4.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_5.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_6.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_7.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_8.